<a href="https://colab.research.google.com/github/lauradekker/Assembly_annotation/blob/master/Ahmad_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# NLP taks to train a model on relations in a sentence
# 1. Upload JSON file into notebook and have data in different variables
# 2. Output should be the relation between the words of the sentences
# 3. Train model on train.json; and validate on dev.json
# 4. Use ROBERTA model for the training -> sequence classification
# 5. Use thigs such as HugginFace, transformers, torch
# 6. Test the model on test.json using an F1 score
# Deadline is June 18th

In [1]:
# Load data
import json
import pandas as pd

#open file
json_file = open('train.json')
#read data into variable
json_df = pd.DataFrame(json.load(json_file))
#load dictionary into a dataframe
#json_df = pd.DataFrame(json_data)
json_df = json_df.dropna()


In [2]:
#print(json_df.shape) #58465 before NAs were dropped, same after

tokens = list(json_df['token'])
#print(tokens)
#print(type(tokens))
#tokens = list(tokens)
print(tokens[0:10])


[['Tom', 'Thabane', 'resigned', 'in', 'October', 'last', 'year', 'to', 'form', 'the', 'All', 'Basotho', 'Convention', '-LRB-', 'ABC', '-RRB-', ',', 'crossing', 'the', 'floor', 'with', '17', 'members', 'of', 'parliament', ',', 'causing', 'constitutional', 'monarch', 'King', 'Letsie', 'III', 'to', 'dissolve', 'parliament', 'and', 'call', 'the', 'snap', 'election', '.'], ['In', '1983', ',', 'a', 'year', 'after', 'the', 'rally', ',', 'Forsberg', 'received', 'the', 'so-called', '``', 'genius', 'award', "''", 'from', 'the', 'John', 'D.', 'and', 'Catherine', 'T.', 'MacArthur', 'Foundation', '.'], ['This', 'was', 'among', 'a', 'batch', 'of', 'paperback', 'Oxford', 'World', "'s", 'Classics', 'that', 'I', 'was', 'given', 'as', 'a', 'reward', 'for', 'reading', 'and', 'commenting', 'on', 'a', 'manuscript', 'for', 'OUP', '.'], ['The', 'latest', 'investigation', 'was', 'authorized', 'after', 'the', 'Supreme', 'Court', 'in', '2007', 'found', 'DCC', 'and', 'its', 'founder', ',', 'Jim', 'Flavin', ',', 

In [ ]:
for i in range(10):
  print(json_data[i])

print(len(json_data))
print(type(json_data))

{'id': '61b3a5c8c9a882dcfcd2', 'docid': 'AFP_ENG_20070218.0019.LDC2009T13', 'relation': 'org:founded_by', 'token': ['Tom', 'Thabane', 'resigned', 'in', 'October', 'last', 'year', 'to', 'form', 'the', 'All', 'Basotho', 'Convention', '-LRB-', 'ABC', '-RRB-', ',', 'crossing', 'the', 'floor', 'with', '17', 'members', 'of', 'parliament', ',', 'causing', 'constitutional', 'monarch', 'King', 'Letsie', 'III', 'to', 'dissolve', 'parliament', 'and', 'call', 'the', 'snap', 'election', '.'], 'subj_start': 10, 'subj_end': 12, 'obj_start': 0, 'obj_end': 1, 'subj_type': 'ORGANIZATION', 'obj_type': 'PERSON', 'stanford_pos': ['NNP', 'NNP', 'VBD', 'IN', 'NNP', 'JJ', 'NN', 'TO', 'VB', 'DT', 'DT', 'NNP', 'NNP', '-LRB-', 'NNP', '-RRB-', ',', 'VBG', 'DT', 'NN', 'IN', 'CD', 'NNS', 'IN', 'NN', ',', 'VBG', 'JJ', 'NN', 'NNP', 'NNP', 'NNP', 'TO', 'VB', 'NN', 'CC', 'VB', 'DT', 'NN', 'NN', '.'], 'stanford_ner': ['PERSON', 'PERSON', 'O', 'O', 'DATE', 'DATE', 'DATE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'ORGANIZATION'

In [ ]:
# RoBERTa Token Classification example

from transformers import AutoTokenizer, RobertaForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
model = RobertaForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")

inputs = tokenizer("HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_token_class_ids = logits.argmax(-1)

predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
print(predicted_tokens_classes)

labels = predicted_token_class_ids
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)


['O', 'ORG', 'ORG', 'O', 'O', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'LOC']


0.01

In [ ]:
# RoBERTa TF Token Classification example

from transformers import AutoTokenizer, TFRobertaForTokenClassification
import tensorflow as tf

#tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
tokenizer = AutoTokenizer.from_pretrained("ydshieh/roberta-large-ner-english")
#
#model = RobertaForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
model = TFRobertaForTokenClassification.from_pretrained("ydshieh/roberta-large-ner-english")
#

#input the token key here?
inputs = tokenizer("HuggingFace is a company based in Paris and New York", add_special_tokens=False, return_tensors="tf")

#with torch.no_grad():
logits = model(**inputs).logits

#predicted_token_class_ids = logits.argmax(-1)
predicted_token_class_ids = tf.math.argmax(logits, axis=-1)
predicted_tokens_classes = [model.config.id2label[t] for t in predicted_token_class_ids[0].numpy().tolist()]
predicted_tokens_classes
#predicted_tokens_classes = [model.config.id2label[t.item()] for t in predicted_token_class_ids[0]]
#predicted_tokens_classes

labels = predicted_token_class_ids
loss = tf.math.reduce_mean(model(**inputs, labels=labels).loss)
round(float(loss), 2)

Some layers from the model checkpoint at ydshieh/roberta-large-ner-english were not used when initializing TFRobertaForTokenClassification: ['dropout_73']
- This IS expected if you are initializing TFRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaForTokenClassification were initialized from the model checkpoint at ydshieh/roberta-large-ner-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForTokenClassification for predictions without further training.


0.01

In [ ]:
# RoBERTa Sequence Classification Example

import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
#inputs

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id] # returns 'optimism' in the example

num_labels = len(model.config.id2label)
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion", num_labels=num_labels)

labels = torch.tensor([1])
loss = model(**inputs, labels=labels).loss
round(loss.item(), 2)

0.08

In [ ]:
# RoBERTa Sequence Classification tokenize the dictionary

import torch
#from transformers import AutoTokenizer, RobertaForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM

#additional input due to suggestions from stackoverflow to error
#tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion", add_prefix_space=True)
#model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base", add_prefix_space=True)
model = AutoModelForMaskedLM.from_pretrained("FacebookAI/roberta-base")

#additional input was put there at the suggestion of the error messages
inputs = tokenizer(tokens, return_tensors="pt", is_split_into_words=True, padding=True, truncation=True)

with torch.no_grad():
    logits = model(**inputs).logits